In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import requests
import random
import yaml
import time
import os
import re

In [ ]:
la_liga = ['https://fbref.com/en/comps/12/', '/schedule/', '-La-Liga-Scores-and-Fixtures', 'La_Liga']
la_liga_2 = ['https://fbref.com/en/comps/17/', '/schedule/', '-Segunda-Division-Scores-and-Fixtures', 'La_Liga_2']
premier_league = ['https://fbref.com/en/comps/9/', '/schedule/', '-Premier-League-Scores-and-Fixtures', 'Premier_League']
championship = ['https://fbref.com/en/comps/10/', '/schedule/', '-Championship-Scores-and-Fixtures', 'Championship']
serie_a = ['https://fbref.com/en/comps/11/', '/schedule/', '-Serie-A-Scores-and-Fixtures', 'Serie_A']
serie_b = ['https://fbref.com/en/comps/18/', '/schedule/', '-Serie-B-Scores-and-Fixtures', 'Serie_B']
ligue_1 = ['https://fbref.com/en/comps/13/', '/schedule/', '-Ligue-1-Scores-and-Fixtures', 'Ligue_1']
ligue_2 = ['https://fbref.com/en/comps/60/','/schedule/','-Ligue-2-Scores-and-Fixtures', 'Ligue_2']
bundesliga = ['https://fbref.com/en/comps/20/', '/schedule/', '-Bundesliga-Scores-and-Fixtures', 'Bundesliga']
bundesliga_2 = ['https://fbref.com/en/comps/33/', '/schedule/', '-2-Bundesliga-Scores-and-Fixtures', 'Bundesliga_2']
belgian_first = ['https://fbref.com/en/comps/37/', '/schedule/','-Belgian-First-Division-A-Scores-and-Fixtures', 'Belgian_First']
belgian_second = ['https://fbref.com/en/comps/69/', '/schedule/','-Belgian-First-Division-B-Scores-and-Fixtures', 'Belgian_Second']
austrian = ['https://fbref.com/en/comps/56/','/schedule/','-Austrian-Bundesliga-Scores-and-Fixtures', 'Austrian_Bundesliga']
bulgarian = ['https://fbref.com/en/comps/67/', '/schedule/', '-Bulgarian-First-League-Scores-and-Fixtures', 'Bulgarian_First']
croatian = ['https://fbref.com/en/comps/63/', '/schedule/', '-1-HNL-Scores-and-Fixtures', 'Croatian_HNL']
czech = ['https://fbref.com/en/comps/66/', '/schedule/', '-Czech-First-League-Scores-and-Fixtures', 'Czech_First']
denmark = ['https://fbref.com/en/comps/50/', '/schedule/', '-Superliga-Scores-and-Fixtures', 'Denmark_Superliga']
greece = ['https://fbref.com/en/comps/27/', '/schedule/', '-Super-League-Greece-Scores-and-Fixtures', 'Super_League_Greece']
hungary = ['https://fbref.com/en/comps/46/', '/schedule/', '-NB-I-Scores-and-Fixtures', 'NB_Hungary']
eredivisie = ['https://fbref.com/en/comps/23/', '/schedule/', '-Eredivisie-Scores-and-Fixtures', 'Eredivisie']
eredivisie_2 = ['https://fbref.com/en/comps/51/', '/schedule/', '-Eerste-Divisie-Scores-and-Fixtures', 'Eredivisie_2']
poland = ['https://fbref.com/en/comps/36/', '/schedule/', '-Ekstraklasa-Scores-and-Fixtures', 'Ekstraklasa_Poland']
portugal = ['https://fbref.com/en/comps/32/', '/schedule/', '-Primeira-Liga-Scores-and-Fixtures', 'Primeira_Liga']
romania = ['https://fbref.com/en/comps/47/', '/schedule/', '-Liga-I-Scores-and-Fixtures', 'Liga_Romania']
russia = ['https://fbref.com/en/comps/30/', '/schedule/', '-Russian-Premier-League-Scores-and-Fixtures', 'Russian_Premier_League']
scottish_1 = ['https://fbref.com/en/comps/40/', '/schedule/', '-Scottish-Premiership-Scores-and-Fixtures', 'Scottish_Premiership']
scottish_2 = ['https://fbref.com/en/comps/72/', '/schedule/', '-Scottish-Championship-Scores-and-Fixtures', 'Scottish_Championship']
serbia = ['https://fbref.com/en/comps/54/', '/schedule/', '-Serbian-SuperLiga-Scores-and-Fixtures', 'Serbian_Superliga']
switzerland = ['https://fbref.com/en/comps/57/', '/schedule/', '-Swiss-Super-League-Scores-and-Fixtures', 'Swiss_Super_League']
turkey = ['https://fbref.com/en/comps/26/', '/schedule/', '-Super-Lig-Scores-and-Fixtures', 'Süper_Lig_Turkey']
mexico = ['https://fbref.com/en/comps/31/', '/schedule/', '-Liga-MX-Scores-and-Fixtures', 'Liga_MX_Mexico']

leagues_championnat = [
    la_liga, la_liga_2, premier_league, championship, serie_a, serie_b, ligue_1, ligue_2, bundesliga, bundesliga_2, 
    belgian_first, belgian_second, austrian, bulgarian, croatian, czech, denmark, greece, hungary, eredivisie, 
    eredivisie_2, poland, portugal, romania, russia, scottish_1, scottish_2 , serbia, switzerland, turkey, mexico
    ]

In [ ]:
def GenerateLinks(leagues):
    
    for league in leagues:

        print("Scraping links for",league[3])

        # Create folder if not exist
        if not os.path.exists("data/"+league[3]):
            os.makedirs("data/"+league[3])

        # Generate links for seasons and add to list 
        links = []
        for i in range(2000,2024):
            links.append(league[0]+str(i)+'-'+str(i+1)+league[1]+str(i)+'-'+str(i+1)+league[2])

        # Get match links for each season
        list_weekday, list_round, list_link, list_season= ([] for i in range(4))

        for link in links:
            match = re.search(r'(\d{4}-\d{4})', link)
            if match: result = match.group(1)
            print("\t### Scraping match urls for season",str(result), "###")

            sleep_time = random.randint(3, 5)
            time.sleep(sleep_time)
            # Make a request to the website
            response = requests.get(link)
            # Parse the HTML content using Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')
          
            # Get all the rows
            for tr in soup.find_all('tr'):

                # find the matchweek tag and get its text
                matchweek_tag = tr.find(attrs={'data-stat': 'gameweek'})
                if matchweek_tag:
                    matchweek = matchweek_tag.text.strip()
                else:
                    matchweek = None

                # find the round tag and get its text
                round_tag = tr.find('th', {'data-stat': 'round'})
                if round_tag:
                    round = round_tag.find('a')
                    if round:
                        round = round.text.strip()
                else:
                    round = 'Regular season'

                # find the match report tag and get its href attribute
                report_tag = tr.find('td', {'data-stat': 'match_report'})
                report_href = report_tag.find('a')['href'] if report_tag and report_tag.find('a') else None

                # Check if it's not a head to head link
                if report_href is not None:

                    if not "stathead" in report_href:
                    
                        # Add to lists
                        list_weekday.append(matchweek)
                        list_round.append(round)
                        list_link.append(report_href)
                        list_season.append(str(result))

        # Create the dataframe
        df = pd.DataFrame(list(zip(list_weekday, list_round, list_link, list_season)), columns =['WK', 'Round', 'Link', 'Season'])
        # Remove None values from link
        df = df.dropna(subset=['Link'])
        # Complete the links
        df['Link'] = ['https://fbref.com' + s for s in df['Link']]
        # Remove duplicates for link column
        df = df.drop_duplicates(subset=['Link'], keep='first')
        # Save to a text file
        df.to_csv("data/"+league[3]+"/"+league[3]+".csv", index=False)

In [ ]:
GenerateLinks(leagues_championnat)